In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
BYTCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 10

In [3]:
pileline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [ ]:
from torch.utils.data import DataLoader

train_set = datasets.MNIST("data",train=True,download=True,transform=pileline)

set_set = datasets.MNIST("data",train=True,download=True,transform=pileline)

100.1%

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


113.5%

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw


75.5%

In [ ]:
train_loader = DataLoader(train_set, batch_size=BYTCH_SIZE,shuffle=True)
test_loader = DataLoader(train_set, batch_size=BYTCH_SIZE,shuffle=True)

In [ ]:
with open("./data/MNIST/raw/train-images-idx3-ubyte","rb") as f:
    file = f.read()

In [ ]:
image1 = [int(str(item).encode('ascii'),16) for item in file[16:16+784]]

In [ ]:
import cv2
import numpy as np
image1_np = np.array(image1,dtype=np.uint8).reshape(28,28,1)
print(image1_np.shape)

In [ ]:
cv2.imwrite("digit.jpg",image1_np)

In [ ]:
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,3)
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def foward(self,x):
        input_size = x.size(0)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2,2)
        
        x = self.conv2(x)
        x = F.relu(x)
        
        x = x.view(input_size, -1)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        
        output = F.log_softmax(x,dim=1)
        
        return output

In [ ]:
model = Digit().to(DEVICE) #定义优化器
optimizer = optim.Adam(model.parameters())

In [ ]:
def train_model(model,device,train_loader,optimizer,epoch): #定义训练方法
   # 模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #部署到DEVICE上去
        data,target = data.to(device),target.to(device)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output,target)
        #找到概率值最大的下标
        #pred = output.max(1,keepdim=True)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print( " Train Epoch : { }\t  Loss : {:.6f}".format(epoch,loss.item()))
            

In [ ]:
def test_model(model,device,test_loader):  #定义测试方法
    model.eval()
    correct = 0.0
    test_loss = 0.0
    with torch.no_grad():#模型已经拿到 不会计算梯度，也不会进行反向传播
        for data,target in test_loader:
            data,target = data.to(device),target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test - Average Loss : {:.4f},Accuracy : {:.3f}\n".format(
            test_loss,100.0 * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1,EPOCHS + 1):  #调用方法 进行训练
    train_model (model, DEVICE, train_loader, optimizer, epoch)
    test_model(model,DEVICE,test_loader)